In [31]:
import pymongo
from dotenv import load_dotenv
import os
load_dotenv(os.path.join('./', ".env"))
from pprint import pprint
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', True)
from IPython.display import display

In [33]:
class Client:
    def __init__(self):
        self.client = pymongo.MongoClient(os.environ.get('MONGO_URI_DIRECT'))
        self.db = self.client['production']
        print('MongoDB Connection Established')
    
    def get_mc_data(self):
        self.collection = self.db['co-mc-sales']
        return self.collection.find({},{
            '_id': 0,
            'administration_price': 1,
            'age': 1,
            'baths': 1,
            'garages': 1,
            'lat': 1,
            'lng': 1,
            'living_area_m2': 1,
            'neighborhood': 1,
            'price': 1,
            'price_m2': 1,
            'propertyType': 1,
            'rooms': 1,
            'stratum': 1,
            'total_area_m2': 1
        })
    
    def get_fr_data(self):
        self.collection = self.db['co-fr-sales']
        return self.collection.find({},{
            '_id': 0,
            'administration_price': 1,
            'age.name': 1,
            'baths': 1,
            'garages': 1,
            'lat': 1,
            'lng': 1,
            'living_area_m2': 1,
            'neighborhood': 1,
            'price': 1,
            'price_m2': 1,
            'propertyType': 1,
            'rooms': 1,
            'stratum': 1,
            'total_area_m2': 1
        })
    
mongo = Client()
mc_data = mongo.get_mc_data()
fr_data = mongo.get_fr_data()
mc_df = pd.DataFrame(list(mc_data))
fr_df = pd.DataFrame(list(fr_data))
print('Data imported as DF')

MongoDB Connection Established


In [35]:
# Concat all data
main_df = pd.concat([mc_df, fr_df], sort=False)

# Filter propertyType
main_df = main_df[main_df['propertyType'].isin(['Casa', 'Apartamento', 'Proyecto', 'Apartaestudio'])]
# Filter stratum
main_df = main_df[main_df['stratum'].isin([1,2,3,4,5,6])]
# Filter age
main_df.loc[(main_df['propertyType'] == 'Proyecto') & (main_df['age'].isnull()), 'age'] = 'Entre 0 y 5 años'
main_df['age'] = main_df['age'].values.astype(str)
conditions = [
    main_df['age'].eq('Entre 0 y 5 años') | main_df['age'].eq('Sobre Plano') | main_df['age'].eq('En Construcción') | main_df['age'].eq('Para Estrenar') | main_df['age'].eq("{'name': 'menor a 1 año'}"),
    main_df['age'].eq('Remodelado') | main_df['age'].eq('Entre 5 y 10 años') | main_df['age'].eq("{'name': '1 a 8 años'}"),
    main_df['age'].eq('Entre 10 y 20 años') | main_df['age'].eq("{'name': '9 a 15 años'}"),
    main_df['age'].eq('Más de 20 años') | main_df['age'].eq("{'name': '16 a 30 años'}") | main_df['age'].eq("{'name': 'más de 30 años'}")                                                                                                                                       
]
choices = ['Menor a 5 años', 'Entre 5 y 10 años', 'Entre 10 y 20 años', 'Mas de 20 años']                                                                                                                                           
main_df['age'] = np.select(conditions, choices, default=np.NaN)


In [37]:
display(main_df.head())

,lat,lng,neighborhood,stratum,propertyType,age,price,price_m2,living_area_m2,total_area_m2,administration_price,garages,rooms,baths
0,4.768007,-74.061005,san-jose-de-bavaria,5.0,Casa,Remodelado,1.850000e+09,4.868421e+06,380.0,380.0,800000.0,4.0,5.0,5.0
1,4.810550,-74.039450,HACIENDA-SAN-SIMON,6.0,Casa,Entre 10 y 20 años,2.950000e+09,9.218750e+06,320.0,320.0,0.0,3.0,4.0,4.0
2,4.647134,-74.055920,CHAPINERO-ALTO-EMAUS,4.0,Apartamento,Más de 20 años,4.600000e+08,6.571429e+06,75.0,70.0,287000.0,1.0,2.0,2.0
3,4.754853,-74.070460,Colina-Campestre,5.0,Casa,Entre 10 y 20 años,6.950000e+08,4.826389e+06,144.0,144.0,500000.0,2.0,3.0,3.0
4,4.718460,-74.031130,Belmira,5.0,Casa,Más de 20 años,1.150000e+09,6.647399e+06,173.0,173.0,456000.0,2.0,3.0,4.0


In [38]:
display(fr_df.head())

,lat,lng,neighborhood,price_m2,stratum,propertyType,living_area_m2,price,rooms,baths,total_area_m2,administration_price,age,garages
0,4.647394,-74.066734,Chapinero,6530612,3,Proyecto,21.24,160000000.0,1.0,1.0,24.50,NaN,NaN,NaN
1,4.647394,-74.066734,Chapinero,6530612,3,Proyecto,21.02,165000000.0,1.0,1.0,24.75,NaN,NaN,NaN
2,4.647394,-74.066734,Chapinero,6530612,3,Proyecto,26.57,185000000.0,1.0,1.0,30.76,NaN,NaN,NaN
3,4.647394,-74.066734,Chapinero,6530612,3,Proyecto,26.18,195000000.0,1.0,1.0,30.90,NaN,NaN,NaN
4,4.647394,-74.066734,Chapinero,6530612,3,Proyecto,29.20,190000000.0,1.0,1.0,33.40,NaN,NaN,NaN
